In [1]:
import re
import numpy as np
import plotly.graph_objects as go

In [2]:
import re

def parse_arteries_exf(file_path):
    """
    Parses an arteries exf file.
    
    Nodes:
      When a "Node:" line is encountered, the next 4 lines are expected:
       - Lines 1-3: The first token in each line is taken as x, y, and z.
       - Line 4: The first token is taken as the radius.
      If any of these lines do not start with a number, the node is recorded as empty.
    
    Mesh1d:
      For each "Element:" line, the parser looks for a "Nodes:" label.
      It then reads the next line to extract the two node IDs connected by that element.
    
    Returns:
      A dictionary with keys:
        "nodes": a list of node dictionaries (each with 'node', 'x', 'y', 'z', and 'r')
        "mesh1d": a list of element dictionaries (each with 'element' and 'nodes' list)
        "empty_nodes": a list of node numbers that did not have valid coordinate data.
    """
    
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    result = {"nodes": [], "mesh1d": [], "empty_nodes": []}
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        # Parse node blocks
        if line.startswith("Node:"):
            # Extract node number
            match = re.match(r"Node:\s*(\d+)", line)
            node_number = int(match.group(1)) if match else None
            coords = []
            valid = True
            
            # Expect next 3 lines to contain x, y, and z values (first token each)
            for j in range(1, 4):
                if i + j < len(lines):
                    coord_line = lines[i+j].strip()
                    parts = coord_line.split()
                    if parts and is_number(parts[0]):
                        coords.append(float(parts[0]))
                    else:
                        valid = False
                        break
                else:
                    valid = False
                    break
            
            # Expect a 4th line for the radius
            if valid and i+4 < len(lines):
                radius_line = lines[i+4].strip()
                parts = radius_line.split()
                if parts and is_number(parts[0]):
                    r = float(parts[0])
                else:
                    valid = False
            else:
                valid = False
            
            if valid and len(coords) == 3:
                result["nodes"].append({
                    "node": node_number,
                    "x": coords[0],
                    "y": coords[1],
                    "z": coords[2],
                    "r": r
                })
                i += 5  # Skip node header + 4 lines
            else:
                result["empty_nodes"].append(node_number)
                i += 1  # Just skip the node header and continue line by line
            continue
        
        # Parse mesh1d connectivity
        if line.startswith("Element:"):
            match = re.match(r"Element:\s*(\d+)", line)
            if match:
                elem_id = int(match.group(1))
                connectivity = []
                found_nodes = False
                i += 1
                while i < len(lines):
                    subline = lines[i].strip()
                    if subline.startswith("Nodes:"):
                        found_nodes = True
                        i += 1  # move to the line with node IDs
                        connectivity = [int(x) for x in lines[i].strip().split()]
                        break
                    else:
                        i += 1
                # We expect each mesh1d element to connect exactly 2 nodes.
                if found_nodes and len(connectivity) == 2:
                    result["mesh1d"].append({
                        "element": elem_id,
                        "nodes": connectivity
                    })
                continue
        
        i += 1

    return result

In [ ]:
file_path = "data/arteries.exf"   # Replace with your actual file path
arteries_data = parse_arteries_exf(file_path)
# Print summary information
print("Number of nodes:", len(arteries_data["nodes"]))
print("Number of mesh1d elements:", len(arteries_data["mesh1d"]))
print("Number of empty nodes:", len(arteries_data["empty_nodes"]))

Number of nodes: 1638
Number of mesh1d elements: 819
Number of empty nodes: 0


In [14]:
arteries_nodes = arteries_data["nodes"]
arteries_mesh = arteries_data["mesh1d"]

In [16]:
# Extract coordinate lists for plotting
x_vals = [node['x'] for node in arteries_nodes]
y_vals = [node['y'] for node in arteries_nodes]
z_vals = [node['z'] for node in arteries_nodes]

# Compute a unified axis range using the min and max of all coordinate values
global_min = min(min(x_vals), min(y_vals), min(z_vals))
global_max = max(max(x_vals), max(y_vals), max(z_vals))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create a 3D scatter plot using Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=x_vals,
    y=y_vals,
    z=z_vals,
    mode='markers',
    marker=dict(size=2)
)])
fig.update_layout(
    title='3D Scatter Plot of SPARC Human Arteries Scaffold',
    scene=dict(
        xaxis=dict(title='X', 
                   range=axis_range
                   ),
        yaxis=dict(title='Y',
                   range=axis_range
                   ),
        zaxis=dict(title='Z',
                   range=[range + 800 for range in axis_range]
                   ),
        aspectmode='cube'  # Ensures equal scale for x, y, z axes
    ),
    width=800,   # Increase width as needed
    height=800   # Increase height as needed
)

fig.show()

In [17]:
# Build a dictionary for quick lookup of node coordinates by node id.
nodes_dict = {node['node']: node for node in arteries_nodes}

# Build lists for the line segments.
line_x = []
line_y = []
line_z = []

for element in arteries_mesh:
    node_ids = element["nodes"]
    # We expect exactly 2 nodes per mesh1d element.
    if len(node_ids) == 2:
        n1, n2 = node_ids
        # Only add the line if both nodes exist.
        if n1 in nodes_dict and n2 in nodes_dict:
            x1, y1, z1 = nodes_dict[n1]['x'], nodes_dict[n1]['y'], nodes_dict[n1]['z']
            x2, y2, z2 = nodes_dict[n2]['x'], nodes_dict[n2]['y'], nodes_dict[n2]['z']
            # Append the two points and a None to break the line.
            line_x += [x1, x2, None]
            line_y += [y1, y2, None]
            line_z += [z1, z2, None]

# Create the node scatter trace.
node_trace = go.Scatter3d(
    x=[node['x'] for node in arteries_nodes],
    y=[node['y'] for node in arteries_nodes],
    z=[node['z'] for node in arteries_nodes],
    mode='markers',
    marker=dict(size=2),
    name='Nodes'
)

# Create the mesh1d line trace.
line_trace = go.Scatter3d(
    x=line_x,
    y=line_y,
    z=line_z,
    mode='lines',
    line=dict(color='green', width=3),
    name='Artery Lines'
)

# Compute unified axis range.
x_vals = [node['x'] for node in arteries_nodes]
y_vals = [node['y'] for node in arteries_nodes]
z_vals = [node['z'] for node in arteries_nodes]
global_min = min(min(x_vals), min(y_vals), min(z_vals))
global_max = max(max(x_vals), max(y_vals), max(z_vals))
abs_max = max(abs(global_min), abs(global_max))
axis_range = [-abs_max, abs_max]

# Create the figure with both traces.
fig = go.Figure(data=[node_trace, line_trace])
fig.update_layout(
    title='3D Scatter Plot of SPARC Human Arteries Scaffold',
    scene=dict(
        xaxis=dict(title='X', range=axis_range),
        yaxis=dict(title='Y', range=axis_range),
        zaxis=dict(title='Z', range=[r + 800 for r in axis_range]),
        aspectmode='cube'
    ),
    width=800,
    height=800
)
fig.show()
